In [1]:
import numpy as np
import pandas as pd
import torch
import glob
import pickle
import matplotlib.pyplot as plt

from interpretation.interpret import compute_importance_score_c_type, compute_importance_score_bias, visualize_sequence_imp
from models.models import CATAC2, CATAC_w_bias

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Sample from peak sequences

In [2]:
with open('../results/peaks_seq.pkl', 'rb') as file:
    seq = pickle.load(file).sequence

seq = seq.sample(10000)

# Compute importance scores

In [4]:
path_model = '../results/train_res/MNLL_model.pkl'
path_seq = '../results/synthetic_results/synthetic_sequences.pkl'

all_c_type = ['Immature', 'Mesenchymal', 'Myoblast', 'Myogenic', 'Neuroblast',
       'Neuronal', 'Somite']
time_point = ["D8", "D12", "D20", "D22"]

#Load the model
model = CATAC_w_bias(nb_conv=8, nb_filters=64, first_kernel=21, 
                      rest_kernel=3, out_pred_len=1024, 
                      nb_pred=4)
        
model.load_state_dict(torch.load(path_model, map_location=torch.device('cpu')))

path_model_bias = "../data/Tn5_NN_model.h5"

#Compute attribution scores
seq, shap_scores, proj_scores = compute_importance_score_bias(model, path_model_bias, seq, device, "Somite", all_c_type, 1)

KeyboardInterrupt: 

In [ ]:
#Save encoded seq + scores
np.savez('../results/encod_seq.npz', seq[:,:4,:])
np.savez('../results/seq_scores.npz', shap_scores[:,:4,:], proj_scores[:,:4,:])

print("Shap scores saved!")

In [ ]:
seq = np.load('../results/encod_seq.npz')["arr_0"]
shap_scores = np.load('../results/seq_scores.npz')
proj_scores = shap_scores['arr_1']; shap_scores = shap_scores['arr_0']

# Visualize few examples

In [ ]:
visualize_sequence_imp(proj_scores[[73],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[1266],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[563],:4,:] ,0, 4096)

# Use TF-modisco to find TFBS
Following tutorial at: https://github.com/jmschrei/tfmodisco-lite/blob/main/examples/ModiscoDemonstration.ipynb

In [ ]:
!modisco motifs -s  ../results/encod_seq.npz -a  ../results/seq_scores.npz -n 2000 -o modisco_results.h5

In [ ]:
!modisco report -i modisco_results.h5 -o report/

In [ ]:
from IPython.display import HTML
HTML('report/motifs.html')

# Run TOMTOM on modisco results

In [ ]:
!modisco report -i modisco_results.h5 -o report/TOMTOM -s report/TOMTOM -m ../data/JASPAR_motif.txt

In [ ]:
from IPython.display import HTML

HTML('report/motifs.html')